<center><img src="RENCI-Official-Logo-transparent.png" width=400 height=100 />

<font size="5">
    <center>
Timeseries Extraction Demonstration Notebook from the RENCI/NOAA 43-year reanalysis
        
    Version 0.4, 12 Sep 2022
    Written by Jim M. McManus, Jeffrey L Tilson, and Brian O. Blanton, RENCI
    Funded by NOAA
  
</font>

<font size="4">

The Renaissance Computing Institute, in collaboration with NOAA's ..., is computing a high-resolution coastal reanalysis dataset using the ADCIRC storm surge, wind-wave, and tide model, and the ECMWF's ERA5 atmospheric reanalisys, spanning the period 1979-2021.  The resulting datasets are in the native ADCIRC/netCDF format, which can complicate working with the large datasets due to the triangular grid used by ADCIRC.  This notebook demostrates the use of a Python package that provides easy access to the datasets for water level and bulk wind wave parameters at user-specified lon/lat points. 
    
Project documentation is available at https://renci.github.io/edsreanalysisdoc/.
</font>
<hr>

## User inputs
The demonstration needs 3 inputs from the user, specified in the box below.
1. Upload a csv file of lon, lat coordinates, with a header line that names the coordinate columns with "lon" and "lat".  The order does not matter. There are 4 example/test files in the testdata directory.
 * **NOAA_Stations_141.csv** contains 141 NOAA stations.  
 * **hsofs_200grid_lonlat.csv** contains a random selection of 200 nodes from the NOAA HSOFS ADCIRC grid.
 * **1point.csv** contains 1 point
 * **hsofs_2grid_lonlat.csv** contains 2 points, one "on land" and the other in water.  The process for the first point will result in all NaNs for the returned timeseries.
2. Specify the variable to extract from the Reanalysis dataset.  The currently available variables are 'water level', 'wave height', 'wave period', 'wave direction'
3. Specify the years to extract using the slider.  The year range is inclusive.

Then, click the Submit button.

### Import/exec the interface package

In [ ]:
from reAnalysisInterface import demoInterface
d = demoInterface()

In [ ]:
# Access DataFrame of data extracted for stations, outside of widgets
print(d.df_product_data.head())

d.df_product_data.plot()